# Tensorboard

TensorBoardはEdwardのプログラムの理解、デバッグ、最適化を簡単にするための一体化した可視化ツールを提供しています。
[tensorflow.org](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard)
によると"TensorFlow graphを可視化したり、グラフを実行した時の定量的な計量をプロットしたり、画像のような付加的なデータを表示するのに使うことができます"

Jupyterでのチュートリアルの原文は[ここ](http://nbviewer.jupyter.org/github/blei-lab/edward/blob/master/notebooks/tensorboard.ipynb)にあります。

![a](http://edwardlib.org/images/tensorboard-scalars.png)

TensorBoardを使うにはまず推測中のログを保存するためのディレクトリを指定しなければいけません。例として推測を手動で行う場合

In [ ]:
inference.initialize(logdir='log')

を呼びます。全てのinference.run()を捕捉しようとする場合logdirを引数に含めます。推測処理が走るとの現在のディレクトリのlog/ 以下に結果が出力されます。
コマンドラインではディレクトリを指定してTensorBoardを実行するには

In [ ]:
tensorboard --logdir=log/

とします。このコマンドはTensorBoardにアクセスするためのwebアドレスを返します。デフォルトではhttp://localhost:6006　です。
正しく動いた場合には上記の画像のように見えるでしょう。

準備完了です！

さらなる手順は　TensorBoardの命名を初期化するために行なわれます。
特に計算グラフないのテンソルとランダム変数の名前を設定してすることでしょう
完全な例を提供するためにここでは隠れた構造をラベルの付いた例$\{(x_n, y_n)\}$から推測する教師あり学習のチュートリアルを拡張します。


# Data

訓練と4040個のデータ点をシミュレートします。データは入力$ \mathbf{x}_n\in\mathbb{R}^{5}$と出力$y_n\in\mathbb{R}y$です。
これらは正規分布のノイズが加わった線形の関係にあります。

In [ ]:
def build_toy_dataset(N, w):
  D = len(w)
  x = np.random.normal(0.0, 2.0, size=(N, D))
  y = np.dot(x, w) + np.random.normal(0.0, 0.01, size=N)
  return x, y

ed.set_seed(42)

N = 40  # number of data points
D = 5  # number of features

w_true = np.random.randn(D) * 0.5
X_train, y_train = build_toy_dataset(N, w_true)
X_test, y_test = build_toy_dataset(N, w_true)

# Model

ベイズ線形回帰モデル(Murphy, 2012)を仮定します。
モデルではN個のデータ点$(\mathbf{X},\mathbf{y})=\{(\mathbf{x}_n, y_n)\}$が以下の分布にしたがっていると仮定します。

\begin{aligned} p(\mathbf{w}) &= \text{Normal}(\mathbf{w} \mid \mathbf{0}, \sigma_w^2\mathbf{I}), \\[1.5ex] p(b) &= \text{Normal}(b \mid 0, \sigma_b^2), \\ p(\mathbf{y} \mid \mathbf{w}, b, \mathbf{X}) &= \prod_{n=1}^N \text{Normal}(y_n \mid \mathbf{x}_n^\top\mathbf{w} + b, \sigma_y^2).\end{aligned}

線形モデルの隠れた変数は重み$\mathbf{w}$とバイアスとも呼ばれる切片bです。$\sigma_w^2,\sigma_b^2$が事前分布の分散で$ \sigma_y^2$が既知の尤度の分散であるとします。
尤度の意味は入力$\mathbf{x}_n$の線形変換から与えられます。

Edwardでモデルを作りましょう。$\sigma_w,\sigma_b,\sigma_y=1$と固定すると

In [ ]:
with tf.name_scope('model'):
  X = tf.placeholder(tf.float32, [N, D], name="X")
  w = Normal(loc=tf.zeros(D, name="weights/loc"),
             scale=tf.ones(D, name="weights/scale"),
             name="weights")
  b = Normal(loc=tf.zeros(1, name="bias/loc"),
             scale=tf.ones(1, name="bias/scale"),
             name="bias")
  y = Normal(loc=ed.dot(X, w) + b,
             scale=tf.ones(N, name="y/scale"),
             name="y")

ここでプレースホルダーXを定義しました。推測の間このプレースホルダーにバッチごとのデータの値を渡します。
また名前のスコープ(tf.name_scope)を用いました。これはスコープの名前 ("model/")をそのwithコンテクスト内の全てのテンソルにプレフィックスとしてつけるものです。
全てのランダム変数のパラメータはグループ化された名前システムに基づいて名前づけされることになります。

# 推測

変分推論を使って事後分布を推測します。
変分モデルを重みに関して正規分布で完全に因子分解されたもの指定定義します。そして変分族のグループの名前の別のスコープを加えます。

In [ ]:
with tf.name_scope("posterior"):
  qw_loc = tf.get_variable("qw/loc", [D])
  qw_scale = tf.nn.softplus(tf.get_variable("qw/unconstrained_scale", [D]))
  qw = Normal(loc=qw_loc, scale=qw_scale, name="qw")
  qb_loc = tf.get_variable("qb/loc", [1])
  qb_scale = tf.nn.softplus(tf.get_variable("qb/unconstrained_scale", [1]))
  qb = Normal(loc=qb_loc, scale=qb_scale, name="qb")

カルバックライブラーダイバージェンスを用いて変分推論を走らせます。ここではblack box stochastic gradient
を計算するために５つの隠れた変数をサンプルします(詳細は[KL(q||p)のチュートリアル](http://edwardlib.org/tutorials/klqp)を見てください。)

In [ ]:
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(n_samples=5, n_iter=250, logdir='log/n_samples_5')

オプションとして名前スコープ"inference"を含めることもできます。
これがないとグラフは分割され一つの"inference"自動的にまとめられることはありません。
もし"inference"が追加されれば計算グラフはより組織立って表示されます。

# Criticism

TensorBoardは任意の問題に対して探索的学習や対話的に使うことができます。
上記のいずれかのコマンドでTensorBoardを走らせた後にタブを操作することができます。

以下では上記のコードをハイパーパラメーターn_samplesが違う2つの設定で実行したものとします。
ログのディレクトリはog/n_samples_です。
デフォルトではEdwardは同じ実験の
異なる実行に対してタイムスタンプの押されたサブディレクトリを作るためにTensorBoardのログは適切に組織づけられます。

推測処理の間　log_timestamp=False　と設定することでこれをoffにすることができます。

### TensorBoard Scalars.

![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard-scalars.png)
    

Scalarsはアルゴリズムのイテレーションによって変化するスカラー値の情報を総経過時間, 総経過時間の相対値に対して表示します。
Edwardではタブにはモデルまたは近似族の中の
スカラーのTensorflowの変数(自由なパラメーター)の値が表示されます。


変分推論においては損失関数(loss関数)やその個々の項のような情報を表示できます。
この例ではn_samples=1の場合にはn_samples=5の場合より高い分散を取りがちであるが同じ解に収束する事が示されています。

### TensorBoard Graphs.

![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard-graphs-0.png)
![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard-graphs-1.png)

Graphsはモデル、近似族、推測処理の計算グラフを表示します。
(灰色の)箱は同じ名前スコープの中にまとめられたテンソルを意味します。
グラフの中の名前を整理することでコードを理解しやすくすることができ、最適化できます。

### TensorBoard Distributions

![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard-distributions.png)

DistributionsはイテレーションごとのスカラーでないTensorFlowの変数の分布を表示します。
これらの変数はモデルと近似族の自由なパラメータです。

### TensorBoard Histograms.

![](https://raw.githubusercontent.com/blei-lab/edward/master/docs/images/tensorboard-histograms.png)

HistogramsはDistributionsと同じ情報を３次元ヒストグラムで表示します。

# Acknowledgments（原文)

このチュートリアルの最初のバージョンを書いたSean Kruzelに感謝します。

TensorflowのTensorBoardは[ここ](https://www.tensorflow.org/get_started/summaries_and_tensorboard)にあります。



# References

Murphy, K. P. (2012). Machine learning: A probabilistic perspective. MIT Press.